Prepare the data for human lisbet experiences

In [1]:
import pandas as pd
import os
import h5py
import math
import random
import numpy as np
import json

SEED = 42
random.seed(SEED)

- prepare datasets 90 children (60ASD , 30TD)
	- Test 35 (5 TD, 30ASD)
	- Training 55 (25TD, 30ASD)
	- v1 : child + clinician
	- v2 : child + 0
	- v3 : child + noise
	- v4 : 0 + clinician
	- v5 : noise + clinician

In [4]:
root = r"./datasets/humans"
root = r"C:\Users\chataint\Documents\projet\humanlisbet\datasets\humans"
nbr_test_td = 10
nbr_test_asd = 20
nbr_train_td = 20
nbr_train_asd = 40


In [4]:
filename = os.path.join(root, "humans_test_annoted.h5")
a=h5py.File(filename)

d_TD = []
for key, vh in a.items():
    data = a[key]['keypoints'][()]
    point = (key, {'keypoints':data})
    d_TD.append(point)
a.close()
print(len(d_TD))
print(d_TD[0][1]['keypoints'][0])

35
[0.429775   0.43177552 0.42978385 0.44394844 0.         0.45827187
 0.43378802 0.46226823 0.42157448 0.44602135 0.41140781 0.47456198
 0.45623229 0.45619844 0.45006771 0.46232448 0.         0.51054653
 0.50787917 0.50781389 0.50770417 0.         0.52133611 0.51059931
 0.54585833 0.53503056 0.57303194 0.55403611 0.56485069 0.56482361
 0.60014444 0.60827014 0.63543819 0.        ]


In [ ]:
filename = os.path.join(root, "humans_TD.h5")
a=h5py.File(filename)

d_TD = []
for key, vh in a.items():
    data = a[key]['keypoints'][()]
    point = (key, {'keypoints':data})
    d_TD.append(point)
a.close()
print(len(d_TD))

30


In [120]:
filename = os.path.join(root, "humans_ASD.h5")
a=h5py.File(filename)

d_ASD = []
for key, vh in a.items():
    data = a[key]['keypoints'][()]
    point = (key, {'keypoints':data})
    d_ASD.append(point)
a.close()
print(len(d_ASD))

60


In [121]:
# V1 all
d5 = d_TD + d_ASD

filename = os.path.join(root, "humans_all.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in d5:
        fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])

In [5]:
filename = os.path.join(root, "humans_all.h5")

with h5py.File(filename, 'r') as fd:

    d_5 = []
    for key, vh in fd.items():
        data = fd[key]['keypoints'][()]
        point = (key, {'keypoints':data})
        d_5.append(point)

print(len(d_5))

90


In [6]:
new_d_5 = []
dm = pd.read_excel(os.path.join(root, r"data_mapping.xlsx"), dtype=str)
dm = dm[dm['From']=='Sara'] 

dm = dm.drop_duplicates()


dm['video'] = dm['VCFS_DATABASE_ADMIN 2::Sujet_ID']
dm['diagnosis'] = dm['VCFS_DATABASE_ADMIN 2::Diagnosis']
dm['diagnosis'] = dm['diagnosis'].replace({'Low-Risk':'TD'})
dm['diagnosis'] = dm['diagnosis'].replace({'Normal_Control':'TD'})
dm['diagnosis'] = dm['diagnosis'].replace({'Autism':'ASD'})

dm['diagnosis'] = dm['diagnosis'].astype('category')

dm['diag_int'] = dm['diagnosis'].cat.codes
category_mapping = dict(enumerate(dm['diagnosis'].cat.categories))

for id, vid_dic in d_5:
    vid_dic['diag'] = dm[dm['video']==id]['diag_int'].to_numpy()
    new_d_5.append((id, vid_dic))



In [8]:
len(new_d_5)

90

In [ ]:
filename = os.path.join(root, "humans_annoted.label.json")
with open(filename, 'w') as fd:
    json.dump(dm[['diagnosis', 'video']].to_dict(), fd, indent=4)

with open(os.path.join(root, 'category_mapping.json'), 'w') as fd :
    json.dump(category_mapping, fd)

In [10]:
id_video = [[],[]]
for i in range(len(new_d_5)):
    id_video[new_d_5[i][1]['diag'][0]].append(new_d_5[i][0])

random.shuffle(id_video[0])
random.shuffle(id_video[1])

idx_test = id_video[0][:nbr_test_asd] + id_video[1][:nbr_test_td]
idx_train = id_video[0][- nbr_train_asd:] + id_video[1][- nbr_train_td:]

# idx_train = idx['train']
# idx_test = idx['test']
db_train = []
db_test = []
for vid in new_d_5:
    id_vid = vid[0]
    if id_vid in idx_train:
        db_train.append(vid)
    elif id_vid in idx_test:
        db_test.append(vid)
    else:
        raise NameError(id_vid)

In [13]:
len(db_train)

60

In [14]:
idx = {
    'train':idx_train,
    'test':idx_test
}
with open(os.path.join(root, 'video_id_split_train_test.json'), 'w') as fd:
    json.dump(idx, fd)

In [33]:
with open(os.path.join(root, 'video_id_split_train_test.json'), 'r') as fd:
    idx = json.load(fd)

In [15]:
filename = os.path.join(root, "humans_test_annoted_30.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in db_test:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])

filename = os.path.join(root, "humans_train_annoted_60.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in db_train:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])

In [29]:
import copy

db_test = copy.deepcopy(dtt)
db_train = copy.deepcopy(dtn)

# with 0
# first 17 keypoints are child ? then clinician
foc_test = copy.deepcopy(db_test)
for i in range(len(foc_test)):
    foc_test[i][1]['keypoints'][:, :17] = 0

filename = os.path.join(root, "humans_test_annoted_first17null.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in foc_test:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])


foc_train = copy.deepcopy(db_train)
for i in range(len(foc_train)):
    foc_train[i][1]['keypoints'][:, :17] = 0



filename = os.path.join(root, "humans_train_annoted_first17null.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in foc_train:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])


    
foc_test = copy.deepcopy(db_test)
for i in range(len(foc_test)):
    foc_test[i][1]['keypoints'][:, 17:] = 0

filename = os.path.join(root, "humans_test_annoted_last17null.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in foc_test:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])


foc_train = copy.deepcopy(db_train)
for i in range(len(foc_train)):
    foc_train[i][1]['keypoints'][:, 17:] = 0

filename = os.path.join(root, "humans_train_annoted_last17null.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in foc_train:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])

In [37]:
import copy

db_test = copy.deepcopy(dtt)
db_train = copy.deepcopy(dtn)

# with 0
# first 17 keypoints are child ? then clinician
foc_test = copy.deepcopy(db_test)
for i in range(len(foc_test)):
    random_values = np.random.rand(*foc_test[i][1]['keypoints'][:, :17].shape)
    foc_test[i][1]['keypoints'][:, :17] = random_values

filename = os.path.join(root, "humans_test_annoted_first17null.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in foc_test:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])


foc_train = copy.deepcopy(db_train)
for i in range(len(foc_train)):
    random_values = np.random.rand(*foc_train[i][1]['keypoints'][:, :17].shape)
    foc_train[i][1]['keypoints'][:, :17] = random_values


filename = os.path.join(root, "humans_train_annoted_first17null.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in foc_train:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])




In [42]:
    
foc_test = copy.deepcopy(db_test)
for i in range(len(foc_test)):
    random_values = np.random.rand(*foc_test[i][1]['keypoints'][:, 17:].shape)
    foc_test[i][1]['keypoints'][:, 17:] = random_values

filename = os.path.join(root, "humans_test_annoted_last17null.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in foc_test:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])


foc_train = copy.deepcopy(db_train)
for i in range(len(foc_train)):
    random_values = np.random.rand(*foc_train[i][1]['keypoints'][:, 17:].shape)
    foc_train[i][1]['keypoints'][:, 17:] = random_values
filename = os.path.join(root, "humans_train_annoted_last17null.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in foc_train:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])

In [43]:
foc_train[0][1]['keypoints'][0]

array([0.65269271, 0.65425   , 0.6465625 , 0.        , 0.62973437,
       0.65265104, 0.62057812, 0.        , 0.60225   , 0.        ,
       0.        , 0.64353125, 0.62213021, 0.64352083, 0.62669792,
       0.64961979, 0.        , 0.40098268, 0.95283105, 0.57225961,
       0.3956643 , 0.3300964 , 0.53134674, 0.55933536, 0.00461687,
       0.87604316, 0.19094607, 0.35538287, 0.91598605, 0.01495376,
       0.46085263, 0.05531905, 0.11935571, 0.09569412])

In [21]:
filename = os.path.join(root, "humans_train_annoted.h5")

with h5py.File(filename, 'r') as fd:

    dtn = []
    for key, vh in fd.items():
        data = fd[key]['keypoints'][()]
        point = (key, {'keypoints':data, 'diag':fd[key]['keypoints'].attrs['diag']})
        dtn.append(point)

print(len(dtn))

55


In [5]:
filename = os.path.join(root, "humans_test_annoted.h5")

with h5py.File(filename, 'r') as fd:

    dtt = []
    for key, vh in fd.items():
        data = fd[key]['keypoints'][()]
        point = (key, {'keypoints':data, 'diag':fd[key]['keypoints'].attrs['diag']})
        dtt.append(point)

print(len(dtt))

35


In [7]:
dtt

[('7772',
  {'keypoints': array([[0.429775  , 0.43177552, 0.42978385, ..., 0.60827014, 0.63543819,
           0.        ],
          [0.        , 0.        , 0.        , ..., 0.62730833, 0.91786111,
           0.80925694],
          [0.42972083, 0.42979531, 0.42980104, ..., 0.62184028, 0.65174583,
           0.        ],
          ...,
          [0.55944792, 0.56703125, 0.55788542, ..., 0.        , 0.        ,
           0.        ],
          [0.55943229, 0.56556771, 0.55788021, ..., 0.        , 0.        ,
           0.        ],
          [0.55794792, 0.56551042, 0.55642187, ..., 0.        , 0.        ,
           0.        ]]),
   'diag': array([1], dtype=int8)}),
 ('7947',
  {'keypoints': array([[0.70464688, 0.71078281, 0.70008281, ..., 0.72554444, 0.79891667,
           0.79072083],
          [0.58237578, 0.58389844, 0.57623594, ..., 0.77174306, 0.75269722,
           0.70925139],
          [0.70465   , 0.71078516, 0.70156016, ..., 0.72552778, 0.79893056,
           0.79072222],


In [6]:
dtt[0][1]

{'keypoints': array([[0.429775  , 0.43177552, 0.42978385, ..., 0.60827014, 0.63543819,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.62730833, 0.91786111,
         0.80925694],
        [0.42972083, 0.42979531, 0.42980104, ..., 0.62184028, 0.65174583,
         0.        ],
        ...,
        [0.55944792, 0.56703125, 0.55788542, ..., 0.        , 0.        ,
         0.        ],
        [0.55943229, 0.56556771, 0.55788021, ..., 0.        , 0.        ,
         0.        ],
        [0.55794792, 0.56551042, 0.55642187, ..., 0.        , 0.        ,
         0.        ]]),
 'diag': array([1], dtype=int8)}